<a href="https://colab.research.google.com/github/yds04312/nownow/blob/master/pose_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


알고리즘 순서

- 1.true data 확정 (거북목 길이 (0-1 위치값이 사람마다 다르므로)  데이터 저장요량: 근데 이 기존의 true data  활용하여 추가로 할 것 찾아보기
(기존의 자세 확인 코드와 동일한 루프를 돌리면 좋을듯))

- 2. 기존의 자세확인 코드



<false 자세>

* 화면 상에서 아예 위의 부분 중 한 부분이라도 안보이는 경우 
이 코드를 가장 위에 넣어야할듯. 예를 들어 15,16,17,18 0 만 보이고 나머지 어깨완련 정보(2,1,5)는 검출이 되지 않는 경우 / 반대의 경우도 포함




* 화면 상에 0,15,16,17,18,2,1,5 까지는 기본적으로 보이는 경우

  1.어깨 불균형 : 3&6 의 기울기 (오차범위 설정)

  2.거북목: 0-1 길이 (길이를 얼마나 해야하는지?)

  3.비스듬한 고개: 0-1 기울기 (오차범위)

  4.눈 관련 detect 가 어디까지 가능한지 확인하기
턱을 괜 자세 1&7 / 1&4 (0,17,18) 가 특정 거리만큼 가까워져있는 경우.


# 자세를 세팅하기위한 코드!
초기 웹 시작 시(한번만 저장하고 해당 값을 데이터베이스에 저장할것인지, 아니면 첫 시행시작에 매번 진행할것인지? 

# 기존의 자세 교정 코드

In [ ]:
# 실시간 json파일 읽어서 원하는 부분 불러옴
import math
import json 

with open("/content/drive/MyDrive/Colab Notebooks/test_pose_000000000530_keypoints.json") as json_file:
  keydata = json.load(json_file) 
  jsonArray= keydata.get("people")


  li = []
  for list in jsonArray:
    li.append(list.get("pose_keypoints_2d"))

  
  #li 라는 2차원 배열에 저장된 위치값들  
  #3k=x위치, 3k+1=y위치 3k+2=신뢰구간


  

#아예 필요한 모든 관절이 모니터상에 없는 경우 : 가장 첫번쨰에 떠야할 알람!
#어떤 자세이든 간에 일단 꼭 검출이 되어야 하는 data는 0,1 2,5,15,16,17,18 이 포인트들은 무조건 검출이 되어야함
# 하나라도 0 값이 나온다면 모니터에 어깨와 얼굴이 다 나오도록 자세를 조정해주세요! 라는 알림을 보낸다. 

  all=0
  all=(li[0][0]*li[0][3]*li[0][6]*li[0][15]*li[0][45]*li[0][48]*li[0][51]*li[0][54])

  if all==0 :
    print("모니터에 어깨와 얼굴이 다 나오도록 자세를 조정해주세요!")
  else:
    print ("좋은 자세입니다")
  #이게 if문을 해결못하면 계속 false, 이걸 해결하면 아래의 조건식으로 넘어가게!


 #어깨 불균형 : point 2&5 의 기울기 말고 그냥 y 좌표로 하면 안될까? (오차범위 설정)
  sh_y=0
  sh_y= li[0][7]-li[0][16]

  if  (-10 >=sh_y or sh_y >=10)  :
        print("어깨의 대칭이 안맞아요")
  else :
        print("어깨 자세가 좋아요!")
 
 

 #비스듬한 고개: 0-1  x축 위치 차이가 잘못된 자세의 경우 100이상 차이가 난다. 바른 자세는 거의 차이가 안남 (10 정도?)

  neck_x = li[0][0]-li[0][3]


  if  (-40 >=neck_x or neck_x>=40)  :
        print("고개를 똑바로 하세요!")
  else :
        print("고개 자세가 바르군요")
 

#flask 코드 내부에서 자세가 좋다는 알람이 두 번 이상 간다면 해당 값을 저장한다: 하영이한테 말하기



#거북목: 0-1 #단순히 길이뿐만 아니라 각 관절들의 비율을 찾아야할듯 

  stan_len=150 
  #이때의 stan_len 값은 true 라고 확정된 상태의 목의 길이 값(초기 사용 시 사용자에게 해당 값을 고정할 시간을 마련한다.)
  
  #neck_x 는 바로 위에 있음

  neck_y= li[0][1]-li[0][4]

  new_turtle = math.sqrt((neck_x  ** 2) + (neck_y ** 2))
  stan_ang= 15 / 180
  
  com_turtle= math.cos(stan_ang*math.pi)*stan_len

  if new_turtle < com_turtle :
    print ("거북목 위험 자세입니다! 고개를 턱 쪽으로 당이고 허리를 펴주세요")
  else :
    print("거북목 위험 자세가 아니에요")

 #턱을 괜 자세 1&7 / 1&4 (0,17,18) 가 특정 거리만큼 가까워져있는 경우.
 #이 부분 너무 어려움
 
 
 
  #눈 관련 detect 가 어디까지 가능한지 확인하기 
  #확인 결과: 눈의 이동은 바로 잘 알아채지만 동일 위치에서의 눈동자만의 이동은 잘 인식하지 못하는 것 같다. 


좋은 자세입니다
어깨 자세가 좋아요!
고개 자세가 바르군요
거북목 위험 자세가 아니에요


0부터 25까지 (뒷배경 포함) 
배열은 지금 [0][i]에 저장되어있음! 그렇다면 내가 원하는 부분을 어떻게 추출? 

총 78개의 data들중에서 필요한 지점의 data들을 비교하여 써야함.

0. {0,  "Nose"},   
1. {3,  "Neck"},   
2.{6,  "RShoulder"},//  
3.{9,  "RElbow"},//    
4. {12,  "RWrist"},//    
5.{15,  "LShoulder"},//     
6.{18,  "LElbow"},//    
7.{21,  "LWrist"},//   
8.{24,  "MidHip"},// 
9.{27,  "RHip"},//  
10.{30, "RKnee"},//    
11. {33, "RAnkle"},//     
12.{36, "LHip"},//  
13.{39, "LKnee"},//    
14.{42, "LAnkle"},//     
15.{45, "REye"},//  
16.{48, "LEye"},//   
17.{51, "REar"},// 
18.{54, "LEar"},//   
19.{57, "LBigToe"},//     
20.{60, "LSmallToe"},//     
21.{63, "LHeel"},//     
22.{66, "RBigToe"},//     
23.{69, "RSmallToe"},//     
24.{72, "RHeel"},//     
25.{75, "Background"}
